In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2010, 1, 1)
end_date = datetime(2016, 8, 31)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


DATE_GRANULARITY_MAPPINGLIST["weekly"].reverse()
DATE_GRANULARITY_MAPPINGLIST["monthly"].reverse()

#print(DATE_GRANULARITY_MAPPINGLIST["weekly"])

sql_template = """
select date, sum(count_a) as app_count, sum(sum_e) as sum_e from plproxy.execute_select_nestloop($proxy$ 
    select date, count(*) as count_a, sum(estimate) as sum_e
    from au.app_weekly
    where 
        date ='{date}' and kpi=1
    group by date order by date desc
$proxy$) tpl (date date, count_a bigint, sum_e double precision ) group by date order by date desc ;

"""

for date_str in DATE_GRANULARITY_MAPPINGLIST["weekly"]:
    try:
        result = query(PLPROXY_DSN, sql_template.format(date=date_str))
        print "{},{:.20g},{:.20g}".format(result[0][0],result[0][1],result[0][2])
    except Exception, e:
        print "{},{}".format(date_str, "ERROR")

